!["question"](question.jpg)



between layer 1 and 2:

$$
\begin{bmatrix}
  b_{10} & w_{11} & w_{21} \\
  b_{20} & w_{12} & w_{22} 
\end{bmatrix}
$$

where b = bias and w = weights

$x = [0.1,0.2]$

In [43]:
import numpy as np
from sklearn.datasets import load_iris


input is columns, output is rows 

In [13]:
def Linear(num_nodes_input_layer, num_nodes_output_layer):
    """Dimensions: output is rows, input is columns."""
    weights = np.random.rand(num_nodes_output_layer, num_nodes_input_layer + 1) * 0.01
    # biases = np.ones((weights.shape[0], 1))

    # Stack the original array and the column of ones
    # return np.column_stack((biases, weights ))
    return weights


# Forward propogation to make predictions

!["how to apply the weights"](apply_weights.png)

$$ output = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [3]:
def compute_forward(inputs, layer):
    ## linear combination w0x0 + w1x1 + w2x2
    linear_combinations = []
    # print(inputs, inputs.shape)
    # print(np.ones(inputs.shape[1]))
    inputs = np.c_[ np.ones(inputs.shape[0]), inputs  ]    ## add bias to the inputs
    # print(inputs)
    # print(layer)

    linear_combinations = np.matmul(inputs, layer.T) ## logits or z
    
    # print(linear_combinations)

    sigmoids = 1/(  1 + np.exp( -linear_combinations ) ) ## compute a
    # print(sigmoids)

    return linear_combinations, sigmoids


# def sigmoid(inputs, layer):

#     linear_combinations = np.matmul(inputs, layer.T) ## logits
#     return 1/(  1 + np.exp( -linear_combinations ) ) ## compute a

# Loss function

*m is number of samples*

*k is total number of output units*
$$
 J(θ)= - \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(\frac{1}{1+e^{xθ^T}})_k + (1-y^{(i)}_k)ln(1-(\frac{1}{1+e^{xθ^T}})_k)]} 
$$
$$
 = \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(a^{(L)}_k) + (1-y^{(i)}_k)ln(1-a^{(L)}_k)]}

$$


$$  
output\_sigmoids = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [40]:
def binary_cross_entropy_loss(output_sigmoids, target):
    
    ## m is number of samples
    ## k is total number of output units

    target = target.reshape(-1,1)
    log_sigmoids = np.log(output_sigmoids)
    y_times_log_sigmoids = log_sigmoids * target ## element-wise multiplication

    one_minus_log_sigmoids = np.log(1 - output_sigmoids)
    one_minus_y_times_log_sigmoids = one_minus_log_sigmoids * (1 - target) ## element-wise multiplication
    # print(one_minus_y_times_log_sigmoids)
    
    return sum(sum((y_times_log_sigmoids + one_minus_y_times_log_sigmoids)))/len(target) * -1 ## sum across the output units and sum across the samples
    # for sample in target:
    #     for row in output_sigmoids:
            
    #         print(row, sample)
    #     print("_____________")




# Backpropogation

want: $\frac{∂J(θ)}{∂θ^{(l)}_{ij}}$ for $l=1,2$

for each

$$δ^{(L)} = \begin{bmatrix}
  a^{(L)}_{1} - y_{11} \\
  a^{(L)}_{2} - y_{12} \\
  a^{(L)}_1 - y_{21} \\ 
  a^{(L)}_1 - y_{22} \\ 

\end{bmatrix}
$$

for the last layer <br>
$y_{11}$ is sample $1$, for target $1 $

$$δ^{(L)} * \begin{bmatrix} a^{(L-1)}_{0}  a^{(L-1)}_{1} a^{(L-1)}_2 \end{bmatrix} = \begin{bmatrix}
  \frac{∂J(θ)}{∂θ^{(L-1)}_{10}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{11}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{12}} \\
  \\
  \frac{∂J(θ)}{∂θ^{(L-1)}_{20}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{21}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{22}} \\


\end{bmatrix} $$

slides 37, 50

every other layer I am going $z^{(L)}$ --> $a^{(L-1)}$ --> $z^{(L-1)}$ and I relate a and z using $\frac{1}{1+e^{-z^{(L-1)}}}$

In [41]:
def compute_backpropogation(sigmoids: list, layers: list, y, x):
    """"""
    # print("layers before reversing")
    # print(layers)
    # layers = [hidden_layer1, output_layer]

    layers.reverse()
    # print("layers after reversing")
    # print(layers)
    # sigmoids = [sigmoids2, output_sigmoids]

    sigmoids.insert(0, x)     
    # print("sigmoids before reversing")
    # print(sigmoids)
    sigmoids.reverse()
    # print("sigmoids after reversing")
    # print(sigmoids)
    gradients = []

    y = y.reshape(-1,1)
    # print(y)

    for i, layer in enumerate(layers):
        if i == 0:

            delta = (sigmoids[i] - y)
            # print(delta.shape)
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print("sigmoids for the last layer")
            # print(sigmoids[i])
            # # print("__ __ __ __")
            # print("sigmoids for the previous layer")
            # print(previous_sigmoids)

            ## delta has shape (samples x nodes)
            ## sigmoids has shape (samples x activations + 1)
            ## ex delta = sx2 (s = samples & nodes = 2 )
            ##    sigmoids = sx3 (s = samples & there are 2 sigmoids and 1 to multiple the bias by)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


        else:
            ## the gradients accumulate across the samples so I may need to do
            ## this for each sample then add all the gradients together. 
            # print("_______________________________")
            ## after the first delta, every previous delta is
            ## (delta * (weights from the previous layer)) * sigmoids or x
            delta = np.matmul(delta, layers[i-1][:,1:]) * sigmoids[i] * (1- sigmoids[i])
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print(previous_sigmoids)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


    # print(gradients)
    return gradients

def step(weights: list, gradients: list, lr = 0.01):
    # updated_weights = []
    gradients.reverse()
    for weight, gradient in zip(weights, gradients): ## so the first gradient is the input layers graidents
        yield weight - (gradient * lr)
        # weight = weight + (gradient * lr)
        # updated_weights.append(weight)

    # return updated_weights
    


In [49]:
data = load_iris()
target = data.target[(data.target == 0) | (data.target == 1)]
data = data.data[(data.target == 0) | (data.target == 1)]

indices = np.random.permutation(target.shape[0])
training_idx, test_idx = indices[:int(target.shape[0] * .7)], indices[int(target.shape[0] * .3):]
target_training, target_test = target[training_idx], target[test_idx]
data_training, data_test = data[training_idx], data[test_idx]




In [67]:
### put it all together:
EPOCHS = 100
# x = np.array([[0.1,0.2],
#               [-.04,-0.6], 
#               [0.1,0.2]]) # x1, x2

# # # y = np.array([[1,0],[1,1],[0,1]])
# y = np.array([[0],[1],[0]])


hidden_layer1 = Linear(4,2) 

output_layer = Linear(2,1) 

# print(hidden_layer1, output_layer)
for epoch in range(EPOCHS):

    z2, sigmoids2 = compute_forward(data_training, hidden_layer1)

    z_output, output_sigmoids = compute_forward(sigmoids2, output_layer)

    loss = binary_cross_entropy_loss(output_sigmoids, target_training)


    gradients = compute_backpropogation(sigmoids = [sigmoids2, output_sigmoids], layers = [hidden_layer1, output_layer], y=target_training, x=data_training)
    hidden_layer1, output_layer = step(weights = [hidden_layer1, output_layer], gradients = gradients, lr = 0.005)
    # print(gradients)
    

    print(f"----------------------epoch: {epoch}-----------------------")
    print(f"predictions:\n{(output_sigmoids.reshape(target_training.shape) > 0.5).astype(int)}\nActual:\n{target_training}")
    print(f"error: {loss}")
    print(f"correct prediction rate: {sum( (output_sigmoids.reshape(target_training.shape) > 0.5).astype(int) == target_training ) / len(target_training) * 100}%")
     ## get amount of correct predictions
    # print(output_sigmoids.reshape(target.shape))
    


----------------------epoch: 0-----------------------
predictions:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual:
[1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]
error: 0.6931591064115215
correct prediction rate: 50.0%
----------------------epoch: 1-----------------------
predictions:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual:
[1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]
error: 0.693151632943445
correct prediction rate: 50.0%
----------------------epoch: 2-----------------------
predictions:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 

#### test on unseen data

In [ ]:
z2, sigmoids2 = compute_forward(data_test, hidden_layer1)
z_output, output_sigmoids = compute_forward(sigmoids2, output_layer)

loss = binary_cross_entropy_loss(output_sigmoids, target_test)

print(f"predictions:\n{(output_sigmoids.reshape(target_test.shape) > 0.5).astype(int)}\nActual:\n{target_test}")
print(f"error: {loss}")

print(f"correct predictions rate: {sum( (output_sigmoids.reshape(target_test.shape) > 0.5).astype(int) == target_test ) / len(target_test) * 100 }%")



predictions:
[0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0
 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1]
Actual:
[0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0
 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1]
error: 0.5604930271896323
correct predictions rate: 100.0%


In [ ]:
def forward(inputs):
    inputs = in

array([[0.13692008, 0.4647963 ],
       [0.65395656, 0.00602726]])

In [69]:
import torch
import torch.nn as nn
import torch.optim as optim

In [97]:
hidden_layer1 = nn.Linear(4,2)
output_layer = nn.Linear(2,1)

x = torch.sigmoid(torch.tensor( data ).to(torch.float32))
x = hidden_layer1(x)
x = torch.sigmoid(x)
x = output_layer(x)


In [94]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()


        # self.input = nn.Linear(2,2)
        self.hidden_layer1 = nn.Linear(4,2)
        self.output_layer = nn.Linear(2,1)

    def foward(self, x):
        # print(x)
        # x = self.input(x)
        x = torch.sigmoid(x)
        x = self.hidden_layer1(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)

        return x
    
model = ANN()

for epoch in range(EPOCHS):
    x_vec = torch.tensor( data ).to(torch.float32)
    tensor_target = torch.tensor(target).to(torch.float32)

    # ----- Forward Propagation -----
    output = model(x_vec)

    # ----- Compute Loss -----
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(output, tensor_target.reshape(-1,1))

    # ----- Backpropagation -----
    optimizer = optim.SGD(model.parameters(), lr = 0.005)
    optimizer.zero_grad() ## remove old gradients
    loss.backward()
    optimizer.step()

    print(f"------------------------------epoch {epoch}--------------------------------------")
    print(f"                  loss: {loss}")
    print(f"predictions:\n{(output.reshape(tensor_target.shape) > 0.5).int()}\nActual:\n{tensor_target}")
    print(f"correct prediction rate: {sum( (output.reshape(tensor_target.shape) > 0.5).int() == tensor_target ) / len(tensor_target) * 100}%")
     ## get amount of correct predictions





NotImplementedError: Module [ANN] is missing the required "forward" function